In [1]:
import torch
from torchtext.legacy import data

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy',
                  tokenizer_language = 'en_core_web_sm')
LABEL = data.LabelField(dtype = torch.float)

In [2]:
from torchtext.legacy import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

aclImdb_v1.tar.gz:   0%|          | 115k/84.1M [00:00<01:15, 1.11MB/s]

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:01<00:00, 66.2MB/s]


In [3]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 25000
Number of testing examples: 25000


In [29]:
print(vars(train_data.examples[0]))

{'text': ['With', 'that', 'line', 'starts', 'one', 'silly', ',', 'boring', 'British', 'sci', 'fi', 'film', '.', 'The', 'Great', 'Vorelli', 'from', 'the', 'movie', '"', 'Devil', 'Doll', '"', 'builds', 'a', 'teleportation', 'machine', 'only', 'to', 'have', 'his', 'funding', 'cut', 'off', 'by', 'Blanchard', ',', 'a', 'bearded', 'man', 'who', 'has', 'a', 'thing', 'for', 'bow', 'ties', 'and', 'men', 'with', 'large', 'eye', 'brows', '.', 'When', 'his', 'experiment', 'fails', ',', 'the', 'good', 'doctor', 'learns', 'that', 'Lembach', ',', 'the', 'man', 'who', 'controls', 'all', 'of', 'the', 'grants', 'in', 'the', 'world', ',', 'will', 'be', 'staying', 'in', 'London', 'for', 'a', 'few', 'days', '.', 'He', 'attempts', 'to', 'project', 'himself', 'into', 'the', 'house', 'of', 'Blanchard', 'with', 'the', 'help', 'of', 'his', 'comely', 'lab', 'assistant', ',', 'Sheila', '.', 'Needless', 'to', 'say', 'something', 'goes', 'wrong', 'and', 'he', 'winds', 'up', 'looking', 'like', 'a', 'rat', '.', 'The'

In [5]:
import random

train_data, valid_data = train_data.split(random_state = random.seed(SEED))

In [6]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 17500
Number of validation examples: 7500
Number of testing examples: 25000


In [7]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)

In [8]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 25002
Unique tokens in LABEL vocabulary: 2


In [9]:
print(TEXT.vocab.freqs.most_common(20))

[('the', 202995), (',', 192589), ('.', 166220), ('a', 109823), ('and', 109605), ('of', 100631), ('to', 94086), ('is', 76261), ('in', 61475), ('I', 54572), ('it', 53798), ('that', 49288), ('"', 44123), ("'s", 43366), ('this', 42618), ('-', 36817), ('/><br', 35673), ('was', 35263), ('as', 30534), ('with', 29957)]


In [10]:
print(TEXT.vocab.itos[:10])

['<unk>', '<pad>', 'the', ',', '.', 'a', 'and', 'of', 'to', 'is']


In [11]:
print(LABEL.vocab.stoi)

defaultdict(None, {'neg': 0, 'pos': 1})


In [86]:
BATCH_SIZE = 130

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    device = device)

In [87]:
for batch in train_iterator:
    print(f'train text matrix size: {batch.text[0].size()}')
    print(f'train target vector size: {batch.label.size()}')
    break


for batch in valid_iterator:
    print(f'valid text matrix size: {batch.text[0].size()}')
    print(f'valid target vector size: {batch.label.size()}')
    break
    
for batch in test_iterator:
    print(f'test text matrix size: {batch.text[0].size()}')
    print(f'test target vector size: {batch.label.size()}')
    break

train text matrix size: torch.Size([130])
train target vector size: torch.Size([130])
valid text matrix size: torch.Size([130])
valid target vector size: torch.Size([130])
test text matrix size: torch.Size([130])
test target vector size: torch.Size([130])


In [88]:
import torch.nn as nn

class LSTM(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        
        self.LSTM = nn.LSTM(embedding_dim, hidden_dim)
        
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text, text_len):

        #text = [sent len, batch size]
        
        embedded = self.embedding(text)
        packed = torch.nn.utils.rnn.pack_padded_sequence(embedded, text_len.to('cpu'))
        packed_output, (hidden, cell) = self.lstm(packed)

        #embedded = [sent len, batch size, emb dim]
                
        #output = [sent len, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        assert torch.equal(output[-1,:,:], hidden.squeeze(0))
        
        return self.fc(hidden.squeeze(0))

In [89]:

INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 128
HIDDEN_DIM = 256
OUTPUT_DIM = 1

model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)

In [90]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 3,299,329 trainable parameters


In [91]:
import torch.optim as optim

optimizer = optim.SGD(model.parameters(), lr=1e-3)

In [92]:
criterion = nn.BCEWithLogitsLoss()

In [93]:
model = model.to(device)
criterion = criterion.to(device)

In [94]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [95]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
                
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [96]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [97]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [98]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 12s
	Train Loss: 0.693 | Train Acc: 49.99%
	 Val. Loss: 0.695 |  Val. Acc: 50.64%
Epoch: 02 | Epoch Time: 0m 12s
	Train Loss: 0.693 | Train Acc: 50.03%
	 Val. Loss: 0.695 |  Val. Acc: 49.93%
Epoch: 03 | Epoch Time: 0m 12s
	Train Loss: 0.693 | Train Acc: 50.54%
	 Val. Loss: 0.696 |  Val. Acc: 49.34%
Epoch: 04 | Epoch Time: 0m 12s
	Train Loss: 0.693 | Train Acc: 50.27%
	 Val. Loss: 0.695 |  Val. Acc: 49.37%
Epoch: 05 | Epoch Time: 0m 12s
	Train Loss: 0.693 | Train Acc: 50.08%
	 Val. Loss: 0.695 |  Val. Acc: 49.38%


In [99]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.687 | Test Acc: 54.01%
